In [ ]:
import os
import pydub
import matplotlib.pyplot as plt
import pydub.silence
import pydub.utils
from pydub.utils import make_chunks
from spleeter.separator import Separator

In [ ]:
separator = Separator('spleeter:2stems') 

files = os.listdir("raw_data")
merged_file = []
all_data = []

for file in files:
    if "_" in file:
        first_part_name = file.split("_")[0] #split first part of file name
        
        
        # Check if there is already a similar file in the merged_file list
        matched_file = None
        for merged in merged_file:
            if merged.startswith(first_part_name) and "_" in merged:
                matched_file = merged
                break
        
        if matched_file:
            # Merge the two parts
            voice_1 = pydub.AudioSegment.from_file(os.path.join("raw_data", matched_file))
            voice_2 = pydub.AudioSegment.from_file(os.path.join("raw_data", file))
            result = voice_1 + voice_2
            
            # Export the merged file
            merged_file_path = os.path.join("raw_data", first_part_name )
            result.export(merged_file_path)
            all_data.append(merged_file_path)
            print(f"Merged {matched_file} and {file} into {first_part_name}")
            
            # Remove the matched file from the list
            merged_file.remove(matched_file)
        else:
            # Add the file to the merged_file list
            merged_file.append(file)
    else:
        # Add files without an underscore directly to the merged_file list
        all_data.append(os.path.join("raw_data", file))

Merged amirhossein_1 and amirhossein_2 into amirhossein
Merged bazazzan_1 and bazazzan_2 into bazazzan
Merged nematizade_1 and nematizade_2 into nematizade
Merged nima_1 and nima_2 into nima
Merged rezai_1 and rezai_2 into rezai
Merged valipour_1 and valipour_2 into valipour


In [ ]:

for data in  all_data:
    
    separator.separate_to_file(data, 'spleeter_output')
    vocal_file_path = os.path.join('spleeter_output', os.path.splitext(os.path.basename(data))[0], 'vocals.wav')
    
    audio = pydub.AudioSegment.from_file(vocal_file_path)
    audio = audio.set_sample_width(2)
    audio = audio.set_frame_rate(48000)
    audio = audio.set_channels(1)
    
    file_name = os.path.basename(data).split(".")[0]

    os.makedirs(os.path.join("final_data",file_name), exist_ok=True)


    # audio_array = audio.get_array_of_samples()
    # plt.figure(figsize=(20,2))
    # plt.plot(audio_array)
    # plt.show()

    chunks = pydub.silence.split_on_silence(audio, min_silence_len=3000  , silence_thresh=-40)

    if chunks:  # Ensure chunks is not empty
        result = sum(chunks)
        parts = make_chunks(result, 1000)
        for i, part in enumerate(parts):
          if len(part) >= 1000:
           part.export(os.path.join("final_data",file_name, f"voice_{i}.wav"), format="wav")
           print(f"Processed and exported voice_{i}.wav")
    else:
        parts = make_chunks(audio, 1000)
        for i, part in enumerate(parts):
          if len(part) >= 1000:
           part.export(os.path.join("final_data",file_name, f"voice_{i}.wav"), format="wav")
           print(f"Processed and exported voice_{i}.wav")
       